# Анализ данных рынка недвижимости. Предобработка данных

In [ ]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt

### 1. Загрузка данных и общая информация

In [ ]:
# Считать датасет, столбец с датой публикации объявления сразу преобразовать в datetime
path = 'https://gist.github.com/a-antonchev/f6eeafda405f4fc19d43965d3bcc462e/raw/rent_apartments.csv'
rent = pd.read_csv(path, sep=',', parse_dates=['published'], date_format='%d/%m/%Y')
rent.head(4)

In [ ]:
rent.info()

In [ ]:
# Получить размерность сформированного датасета
rent.shape

In [ ]:
# Значения в столбце rooms (количество комнат в квартире); квартиры с количеством комнат 0 - студии
rent.rooms.value_counts()

In [ ]:
# Значения и их количество в столбце material (материал постройки дома)
rent.material.value_counts()

In [ ]:
# Значения и их количество в столбце build_oldest (новизна дома)
rent.build_oldest.value_counts()

### 2. Обработка дубликатов

В соответствии с техническим заданием проекта необходимо удалить только полные дубликаты строк.

In [ ]:
# Вывести количество строк полных дубликатов
rent.duplicated().sum()

In [ ]:
# Вывести несколько дубликатов для информации
rent[rent.duplicated()][:3]

In [ ]:
# Сохранить дубликаты в файл
rent[rent.duplicated()].to_csv('rent_apartments_duplicates.csv')

In [ ]:
# Удалить полные дубликаты строк
rent = rent.drop_duplicates()

### 3. Отбор столбцов для анализа

In [ ]:
usecols = ['rooms', 'level', 'area', 'price', 'material', 'published', 'city', 'remoute_from_center', 'build_oldest']
rent = rent.loc[:, usecols]
rent[:3]

### 4. Обработка пропусков

In [ ]:
# Получить сводные данные по наличию пропусков данных
rent.isna().sum()

In [ ]:
# В колонке 'material' (материал постройки дома) присутствуют пропуски данных
rent.loc[rent.material.isna()][:3]

In [ ]:
# Оценить масштаб проблемы
# Рассчитаем долю строк с пропущенными значениями к общему количеству строк целевом наборе данных (объявления в Москве с 01.01.2021 по 01.02.2022 включительно)
target = (rent.city == "Москва") & (rent.published >= dt.datetime(2021, 1, 1)) & (rent.published < dt.datetime(2022, 3, 1))
count_all_target_lines = rent.loc[target].shape[0]
count_lines_with_material_nan = rent.loc[target & (rent.material.isna())].shape[0]
prc_ratio_nan = count_lines_with_material_nan * 100.0 / count_all_target_lines
print(f'Доля строк с пропущенными значениями material к общему количеству строк: {prc_ratio_nan:.2f}%')

In [ ]:
# Доля строк с пропущенными значениями составляет 0,24%, принято решение удалить такие строки 
rent = rent.dropna()

### 5. Распаковка столбца `level`

В столбце level указан этаж квартиры и общее количество этажей в доме через прямой слеш '/'. Необходимо разделить значения этажа и этажности по разным столбцам. Считаем, что значение этажа - целое число

In [ ]:
def get_level(s, i):
    """
    Возвращает элемент списка, приведенный к типу int, по заданному индексу.
    При возникновении исключения ValueError возвращает NaN
    Parameters
    ----------
    s : {list}
        Список из которого требуется возвратить элемент
    i : {int}
        Индекс возвращаемого элемента списка
    Returns
    -------
    Элемент списка, преобразованный в int или NaN, при ошибке обработки    
    """
    try:
        return int(s[i])
    except ValueError:
        return np.nan

In [ ]:
rent['level_temp'] = rent['level'].str.split('/') # создать временный столбец, разделив значения в cтолбце 'level' по '/'
rent['level'] = rent['level_temp'].apply(get_level, i=0) # в столбец 'level' записать этаж
rent['level_all'] = rent['level_temp'].apply(get_level, i=1) # в столбец 'level_all' записать общее количество этажей дома
rent = rent.drop(labels=['level_temp'], axis=1)  # удалить временный столбец
rent.isna().sum() # проверить корректность распаковки столбца 'level' (отсутствие значений NaN)

### 6. Замена категорий

In [ ]:
mapping_template = {'new': 'новостройка', 'old': 'старый фонд', 'middle': 'вторичка'}
rent['build_oldest'] = rent['build_oldest'].map(mapping_template)

### 7. Расчет цены аренды за 1 кв. метр

In [ ]:
rent['price_per_meter'] = rent['price'].div(rent['area']).round(0)

### 8. Округление данных

In [ ]:
rent['remoute_from_center'] = rent['remoute_from_center'].round(0)

### 9. Проверка валидности данных

In [ ]:
# Проверить, что этаж не превышает этажности здания
rent.query('level > level_all')

### 10. Фильтрация данных

In [ ]:
# Отфильтровать данные для анализа, оставить только объявления по Москве за период с 01.01.2021 по 01.02.2022 включительно
start_date = dt.datetime(2021, 1, 1)
end_date = dt.datetime(2022, 3, 1)
rent = rent.loc[(rent['published'] >= start_date) & (rent['published'] < end_date) & (rent['city'] == 'Москва')]

In [ ]:
# Вывести размерность датасета после фильтрации
rent.shape

### 11. Удаление выбросов

Визуализируем распределения количественных признаков: level, area, price, remoute_from_center, price_per_meter

In [ ]:
# чтобы вывести распределения на одном графике, нормируем признаки с помощью z-оценки и сформируем новый датафрейм
std = pd.DataFrame()

cols = ['price', 'price_per_meter', 'area', 'level', 'level_all', 'remoute_from_center', 'rooms']

for col in cols:
    m = rent[col].mean()
    s = rent[col].std()
    std[col + '_n'] = (rent[col] - m) / s

std.head(3)

In [ ]:
def show_boxplots(df):
    """
    Визуализация распределения датафрейма с помощью boxplots
    Parameters
    ----------
    df : {pd.DataFrame}
         Датафрейм для визуализации
    Returns
    -------
    Boxplots
    """
    fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(12, 5))
    ax.tick_params(labelsize=9)
    ax.set_xticklabels(df.columns)
    ax.boxplot(df)
    plt.show();

In [ ]:
# визуализируем распределения признаков
show_boxplots(std)

In [ ]:
# удалим выбросы в наборах данных price_n и price_per_meter_n и заново визуализируем распределения
cut_outliers = (std.price_n <= 40) & (std.price_per_meter_n <= 50)
std = std.loc[cut_outliers]
show_boxplots(std)

Графики распределения количественных данных level, area, price, remoute_from_center, price_per_meter не симметричны, скошены вправо, что свидетельствет о наличии на рынке недвижимости Москвы высотных домов, домов со значительной удаленностью от центра и квартир премиум-сегмента (квартиры большой площади, с высокой арендной платой).\
Чтобы минимизировать влияние выбросов, но при этом сохранить общую картину рынка, принято решение удалить записи со значениями, которые превышают:
- по площади (area) 260 кв. м 
- по арендной плате (price) - 650 тыс. руб.
- цене аренды за 1 кв. метр (price_per_meter) - 6000 руб.
- по расстоянию от центра (remoute_from_center) - 40 км
- по количеству этажей дома (level_all) - 90
- по количеству комнат (rooms) - 6

In [ ]:
# сформировать фильтр и отсечь 'экстремальные' значения
cut_outliers =(rent.area <= 260) & (rent.price <= 650_000) & (rent.price_per_meter <= 6_000) & (rent.remoute_from_center <= 40) & (rent.level_all <= 90) & (rent.rooms <= 6)
rent = rent.loc[cut_outliers]

In [ ]:
rent.reset_index(drop=True, inplace=True)

In [ ]:
# вывести размерность итогового датасета
rent.shape

In [ ]:
fig, ax = plt.subplots(nrows=2, ncols=3, figsize=(15, 6))
labels = [['Этажность дома', 'Количество этажей', 'Площадь квартиры, кв.м'], ['Цена аренды, руб.', 'Цена аренды за 1 кв. метр, руб.', 'Расстояние от центра, км']]
columns =[['level_all', 'level', 'area'], ['price', 'price_per_meter', 'remoute_from_center']]
for i in range(2):
    for j in range(3):
        ax[i][j].set_xlabel(labels[i][j], fontsize=9)
        ax[i][j].set_ylabel('Количество квартир', fontsize=9)
        ax[i][j].tick_params(axis='both', labelsize=8)
        ax[i][j].hist(rent[columns[i][j]], bins=20, color='#394075')

plt.show();

### 12. Переименование столбцов

In [ ]:
columns={'rooms': 'Количество комнат',
         'level': 'Этаж',
         'area': 'Площадь',
         'price': 'Цена аренды',
         'material': 'Конструктив дома',
         'published': 'Дата публикации',
         'city': 'Город',
         'remoute_from_center': 'Расстояние от центра',
         'build_oldest': 'Тип недвижимости',
         'level_all': 'Этажность',
         'price_per_meter': 'Цена аренды за кв.метр'}
rent = rent.rename(columns=columns)
rent[:3]

### 13. Сохранение датасета в файл

In [ ]:
rent.to_csv('rent_apartament_for analysis.csv', index_label='id')